In [1]:
import matplotlib as mpl
from scipy.io import netcdf
import matplotlib
#from numpy import *
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import datetime
from datetime import datetime, timedelta
import cartopy.crs as ccrs
import matplotlib.colors as mcolors
import xarray
import metpy.interpolate
#%matplotlib inline

In [2]:
#Check memory: ps aux  | awk '{print $6/1024 " MB\t\t" $11}'  | sort -nr | head -10

In [5]:
# creates lists for each variable with file names of each 6 hour timestep for specified number of days

dstart = datetime(2015, 1, 1, 0, 0, 0)
ndays = 1096+365 # 3 years including 1 leap year
nfiles = 4
fnameu = []
fnamev = []
fnamephi = []

for t in np.arange(0,ndays):
    
    for i in np.arange(0, nfiles):

        datefmt = '{:%Y%m%dT%H%M}'.format(dstart)
        dateyear = '{:%Y}'.format(dstart)
        dateday = '{:%m}'.format(dstart)
    
        fnameu.append('/g/data/cj37/BARRA/BARRA_R/v1/forecast/prs/wnd_ucmp/'+dateyear+'/'+dateday+'/wnd_ucmp-fc-prs-PT1H-BARRA_R-v1-'+datefmt+'Z.sub.nc')
        fnamev.append('/g/data/cj37/BARRA/BARRA_R/v1/forecast/prs/wnd_vcmp/'+dateyear+'/'+dateday+'/wnd_vcmp-fc-prs-PT1H-BARRA_R-v1-'+datefmt+'Z.sub.nc')
        fnamephi.append('/g/data/cj37/BARRA_R/v1/forecast/prs/geop_ht/'+dateyear+'/'+dateday+'/geop_ht-fc-prs-PT1H-BARRA_R-v1-'+datefmt+'Z.sub.nc')

        dstart =  dstart + timedelta(hours=6)
    
    dstart =  dstart + timedelta(hours=0)
    


In [6]:
chunks = {'time': 1, 'level': -1} #god knows what this does but seems to work
u = xarray.open_mfdataset(fnameu, chunks = chunks, engine = 'netcdf4').wnd_ucmp
v = xarray.open_mfdataset(fnamev, chunks = chunks, engine = 'netcdf4').wnd_vcmp
# height = xarray.open_mfdataset(fnamephi, chunks = chunks).geop_ht

# topog = xarray.open_dataset('Downloads/topog-an-slv-PT0H-BARRA_R-v1-20150207T0000Z.nc')
# topog = topog.topog

In [ ]:
nj = 40
ni = 83
nt = ndays*24

jstart = 241
jend = jstart + nj
istart = 690
iend = istart + ni
pstart = 28
pend = 37
plevels = pend-pstart

us = u.isel


us = u[:, pstart:pend,jstart:jend,istart:iend]
vs = v[:,pstart:pend,jstart:jend,istart:iend]
heights = height[:,pstart:pend,jstart:jend,istart:iend] #10 pressure levels
topog = topog[jstart:jend,istart:iend]

lon2d, lat2d = np.meshgrid(us.longitude.data, us.latitude.data)

# Calculate the wind and height
wind = np.sqrt(us**2 + vs**2)
height = heights #geopotential height at 6 pressure levels - geoheight already in metres in BARRA (no need to divide by gravity)

In [ ]:
wind = windnp

In [ ]:
print('height (MB): ', height.nbytes/1024**2)
print('us (MB): ', us.nbytes/1024**2)
print('vs (MB): ', vs.nbytes/1024**2)
print('wind (MB): ', wind.nbytes/1024**2)

In [ ]:
# We test on time=0 only for speed. Add a loop on time. Need to compute to use metpy. It doesn't work lazily. 
# But that's not a problem for memory with 1 timestep
heights = xarray.DataArray(np.empty([nt,plevels,nj,ni]), dims = ("time","isosurf","latitude","longitude")) #create empty xarray dataframe
winds = xarray.DataArray(np.empty([nt,plevels,nj,ni]), dims = ("time","isosurf","latitude","longitude") )
heights_std = np.arange(100.0,1200.0,100.0) # Up to 1200m
agl = xarray.DataArray(np.empty([nt,plevels,nj,ni])) # agl = above ground level

# Define coordinates for the data on isosurfaces
coords_isosurf={
    winds.dims[0]:winds.coords[winds.dims[0]],
    "isosurf":heights_std,
    winds.dims[2]:winds.coords[winds.dims[2]],   # latitude name and dimension
    winds.dims[3]:winds.coords[winds.dims[3]]    # longitude name and dimension
}
wsprof_interp = xarray.DataArray(np.empty([winds.shape[0],heights_std.shape[0],winds.shape[2],winds.shape[3]]), 
                                 dims=(winds.dims[0],"isosurf",winds.dims[2],winds.dims[3]),
                                 coords=coords_isosurf) #numpy.empty([heights_std.shape[0],wind0.shape[1],wind0.shape[2]])

for t in np.arange(0,6*nfiles*ndays):
    
    heights[t,...]=height[t,...].compute() #geopotential height at 9 pressure levels
    winds[t,...]=wind[t,...].compute() #windspeed at 9 pressure levels
    agl[t,...]=heights[t,...] - topog #height above ground level (topography accounted for)

    
    # Interpolate for all locations at once
    for niso,isosurf in enumerate(heights_std):
        #windspeed profile above ground level
        wsprof_interp[t,niso,...] = metpy.interpolate.interpolate_to_isosurface(agl.data[t,...], winds.data[t,...], isosurf)


In [ ]:
print('heights (MB): ', heights.nbytes/1024**2)
print('winds (MB): ', winds.nbytes/1024**2)
print('agl (MB): ', agl.nbytes/1024**2)

In [ ]:
# data = xarray.open_dataset('LLJ_sept2016.nc')
# topog = xarray.open_dataset('Downloads/topog-an-slv-PT0H-BARRA_R-v1-20150207T0000Z.nc')
# # wsprof_interp = data.wsprof_interp
# # lon2d = data.lon2d
# # lat2d = data.lat2d
# topog = topog.topog
# topog = topog[jstart:jend,istart:iend]

In [ ]:
# profmax_ind = np.argmax(wsprof_interp.data[:,1:10,...], axis=1) + 1 # must add 1 if starting at index = 1

# profmax_ind_above = profmax_ind + 1
# profmax_ind_below = profmax_ind - 1      
    

In [ ]:
# profmax_ind_above = xarray.DataArray(profmax_ind_above, dims=("time","latitude","longitude"))  
# profmax_ind_below = xarray.DataArray(profmax_ind_below, dims=("time","latitude","longitude"))
# profmax_ind = xarray.DataArray(profmax_ind, dims=("time","latitude","longitude"))


In [ ]:
# profmax_ind_above.max()

In [ ]:
# wsabove = wsprof_interp.isel(isosurf = profmax_ind_above) #windspeed above max evaluated on isosurf axis
# profmax = wsprof_interp.isel(isosurf = profmax_ind)
# wsbelow = wsprof_interp.isel(isosurf = profmax_ind_below) #windspeed below

# print(wsabove.nbytes/1000**2)

In [ ]:
# # variable criteria algorithm

# fixed_criteria = 1
# variable_criteria = 0.1 # windspeed above and below is more than x% below the maxima
# percent =  1 - variable_criteria 

# speed_crit = profmax > 7
# below_crit = wsbelow <= percent * profmax
# above_crit = wsabove <= percent * profmax

# jetbinary_variable = (speed_crit & below_crit & above_crit) * 1

# # fixed criteria algorithm
# jetbinary_fixed = ((profmax - wsabove >= fixed_criteria) & (profmax - wsbelow >= fixed_criteria)) * 1


In [ ]:
# totaljets_variable = np.zeros([nj,ni])
# totaljets_fixed = np.zeros([nj,ni])
# for t in np.arange(0,nt):
#     totaljets_variable = totaljets_variable + jetbinary_variable[t,...]
#     totaljets_fixed = totaljets_fixed + jetbinary_fixed[t,...]


In [ ]:
# def contourplot(profile):#, **latstart, **latend, **lonstart, **lonend):
#     levels = np.arange(0,profile.max(),1)
#     plt.figure(figsize=(12, 12))
#     src = ccrs.PlateCarree()
#     ax = plt.axes(projection=src)
#     ax.coastlines('50m', linewidth=1)
#     cmap=matplotlib.cm.get_cmap('RdPu')
#     cax,kw = matplotlib.colorbar.make_axes(ax,location='bottom',pad=0.05,shrink=0.7)
#     lh = ax.gridlines(xlocs=range(131,171,1), ylocs=range(-39,-27,1),draw_labels=True)
#     lh.top_labels = False
#     lh.right_labels = False
#     #lon = lon2d.isel(dim_0 = slice(31,33), dim_1= slice(80,82))
#     #lat = lat2d.isel(dim_0 = slice(31,33), dim_1= slice(80,82))
#     ch = ax.contourf(lon2d, lat2d, profile, levels, extend = 'both',cmap=cmap)
#     out=plt.colorbar(ch,cax=cax,**kw)
#     out.set_label('Frequency',size=10)

In [ ]:
# s = totaljets_variable#.isel(latitude = slice(31,33), longitude= slice(80,82))
# contourplot(s)


In [ ]:
# def gridpoints(latitude, longitude):
#     lat = np.linspace(lat2d.min(), lat2d.max(), nj) #min, max lat
#     lon = np.linspace(lon2d.min(),lon2d.max(),ni) #min, max lon
#     abs_val_lat, abs_val_lon = np.abs(lat - latitude), np.abs(lon - longitude)
#     lat_grid, lon_grid  =  abs_val_lat.argmin(), abs_val_lon.argmin()
#     return lat_grid, lon_grid
#     print('lat,lon', lat_grid,',', lon_grid)
# gridpoints(-35.5,146.75)
# gridpoints(-36,146.5)

In [ ]:
wsprof_interp.nbytes/1024**2

### Save to net cdf file

In [ ]:
# Write to net cdf file
lon2d = xarray.DataArray(lon2d, name = 'lon2d')
lat2d = xarray.DataArray(lat2d, name = 'lat2d')
wsprof_interp = xarray.DataArray(wsprof_interp, name = 'wsprof_interp')

data = xarray.merge([lon2d, lat2d, wsprof_interp], compat = 'override')
data.to_netcdf('/g/data/v46/lh2344/LLJ_2015_17.nc')

### Concatenate multiple years

In [ ]:
t1517 = np.arange(dstart, datetime(2015,1,1,0,0,0) + timedelta(hours=nt), timedelta(hours=1)).astype(datetime)
t18 = np.arange(datetime(2018,1,1,0,0,0), datetime(2018,1,1,0,0,0) + timedelta(hours=(365*24)), timedelta(hours=1)).astype(datetime)

In [ ]:
LLJ_2015_17 = xarray.open_dataset('/g/data/v46/lh2344/LLJ_2015_17.nc')
LLJ_2015_17["time"] = t1517
LLJ_2018 = xarray.open_dataset('/g/data/v46/lh2344/LLJ_2018.nc')
LLJ_2018["time"] = t18
LLJ_2015_18 = xarray.concat([LLJ_2015_17, LLJ_2018], dim = "time")

In [ ]:
LLJ_2015_18.to_netcdf('/g/data/v46/lh2344/LLJ_2015_18.nc')